In [1]:
import tensorflow as tf
from tensorflow import keras

In [5]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()

x_train_full, x_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target)
x_train, x_val, y_train, y_val = train_test_split(
    x_train_full, y_train_full)

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_val = scaler.transform(x_val)
x_test = scaler.transform(x_test)

x_train_a, x_train_b = x_train[:, :5], x_train[:, 2:]
x_val_a, x_val_b = x_val[:, :5], x_val[:, 2:]
x_test_a, x_test_b = x_test[:, :5], x_test[:, 2:]

In [6]:
input_a = keras.layers.Input(shape=[5], name="wide_input")
input_b = keras.layers.Input(shape=[6], name="deep_input")
h1 = keras.layers.Dense(30, activation="relu")(input_b)
h2 = keras.layers.Dense(30, activation="relu")(h1)
concat = keras.layers.Concatenate()([input_a, h2])
output = keras.layers.Dense(1, name="main_output")(concat)
aux_output = keras.layers.Dense(1, name="aux_output")(h2)

model3 = keras.Model(inputs=[input_a, input_b], outputs=[output, aux_output])
model3.compile(loss=["mse", "mse"], loss_weights=[0.9, 0.1], optimizer="sgd")

In [7]:
hist = model3.fit((x_train_a, x_train_b), y_train, epochs=20,
          validation_data=((x_val_a, x_val_b), y_val))

Epoch 1/20
363/363 [==============================] - 1s 1ms/step - loss: 1.5195 - main_output_loss: 1.3906 - aux_output_loss: 2.6795 - val_loss: 0.6717 - val_main_output_loss: 0.5955 - val_aux_output_loss: 1.3573
Epoch 2/20
363/363 [==============================] - 0s 671us/step - loss: 0.7660 - main_output_loss: 0.7110 - aux_output_loss: 1.2609 - val_loss: 0.5920 - val_main_output_loss: 0.5308 - val_aux_output_loss: 1.1427
Epoch 3/20
363/363 [==============================] - 0s 667us/step - loss: 0.5867 - main_output_loss: 0.5337 - aux_output_loss: 1.0641 - val_loss: 0.5366 - val_main_output_loss: 0.4854 - val_aux_output_loss: 0.9972
Epoch 4/20
363/363 [==============================] - 0s 653us/step - loss: 0.5672 - main_output_loss: 0.5240 - aux_output_loss: 0.9562 - val_loss: 0.5071 - val_main_output_loss: 0.4658 - val_aux_output_loss: 0.8789
Epoch 5/20
363/363 [==============================] - 0s 655us/step - loss: 0.5095 - main_output_loss: 0.4744 - aux_output_loss: 0.8255 - 

In [8]:
model3.save("my_keras_model.h5")

케라스는 HDF5 포맷을 사용하여 모델 구조와 층의 모든 모델 파라미터를 저장함.  
또한 (하이퍼파라미터와 현재 상태를 포함하여) 옵티마이저도 저장함.

In [9]:
loaded_model = keras.models.load_model("my_keras_model.h5")

In [14]:
loaded_model.evaluate((x_test_a, x_test_b), y_test)

162/162 [==============================] - 0s 646us/step - loss: 0.3575 - main_output_loss: 0.3438 - aux_output_loss: 0.4808


[0.35750168561935425, 0.3438011109828949, 0.4808073341846466]

---
## 콜백 사용하기
fit()메서드의 **callbacks** 매개변수를 사용하여 학습의 시작이나 끝에 호출할 객체리스트를 지정할 수 있음.  
또는 에포크의 시작이나 끝, 각 배치 처리 전후에 호출할 수도 있음.

In [17]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("my_keras_model.h5")
hist = model3.fit((x_train_a, x_train_b), y_train, epochs=10,
                callbacks=[checkpoint_cb])

Epoch 1/10
363/363 [==============================] - 0s 448us/step - loss: 0.3692 - main_output_loss: 0.3556 - aux_output_loss: 0.4912
Epoch 2/10
363/363 [==============================] - 0s 453us/step - loss: 0.3698 - main_output_loss: 0.3570 - aux_output_loss: 0.4844
Epoch 3/10
363/363 [==============================] - 0s 451us/step - loss: 0.3651 - main_output_loss: 0.3523 - aux_output_loss: 0.4801
Epoch 4/10
363/363 [==============================] - 0s 480us/step - loss: 0.3617 - main_output_loss: 0.3491 - aux_output_loss: 0.4759
Epoch 5/10
363/363 [==============================] - 0s 461us/step - loss: 0.3629 - main_output_loss: 0.3509 - aux_output_loss: 0.4714
Epoch 6/10
363/363 [==============================] - 0s 453us/step - loss: 0.3583 - main_output_loss: 0.3460 - aux_output_loss: 0.4687
Epoch 7/10
363/363 [==============================] - 0s 456us/step - loss: 0.3550 - main_output_loss: 0.3433 - aux_output_loss: 0.4600
Epoch 8/10
363/363 [============================

In [18]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("my_keras_model.h5", 
                                               save_best_only=True)
hist = model3.fit((x_train_a, x_train_b), y_train, epochs=10,
                callbacks=[checkpoint_cb])

model = keras.models.load_model("my_keras_model.h5")

Epoch 1/10
363/363 [==============================] - 0s 473us/step - loss: 0.3486 - main_output_loss: 0.3375 - aux_output_loss: 0.4482
Epoch 2/10
363/363 [==============================] - 0s 484us/step - loss: 0.3491 - main_output_loss: 0.3383 - aux_output_loss: 0.4470
Epoch 3/10
363/363 [==============================] - 0s 458us/step - loss: 0.3479 - main_output_loss: 0.3372 - aux_output_loss: 0.4442
Epoch 4/10
363/363 [==============================] - 0s 450us/step - loss: 0.3459 - main_output_loss: 0.3353 - aux_output_loss: 0.4419
Epoch 5/10
363/363 [==============================] - 0s 448us/step - loss: 0.3450 - main_output_loss: 0.3345 - aux_output_loss: 0.4397
Epoch 6/10
363/363 [==============================] - 0s 453us/step - loss: 0.3428 - main_output_loss: 0.3324 - aux_output_loss: 0.4366
Epoch 7/10
363/363 [==============================] - 0s 489us/step - loss: 0.3457 - main_output_loss: 0.3355 - aux_output_loss: 0.4373
Epoch 8/10
363/363 [============================

**save_best_only=True** 로 지정하면 최상의 검증 세트 점수에서만 모델을 저장함.

In [22]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=5,
                                                 restore_best_weights=True)
hist = model3.fit((x_train_a, x_train_b), y_train, epochs=200,
                callbacks=[checkpoint_cb, early_stopping_cb])

Epoch 1/200
363/363 [==============================] - 0s 514us/step - loss: 0.2725 - main_output_loss: 0.2674 - aux_output_loss: 0.3181
Epoch 2/200
363/363 [==============================] - 0s 521us/step - loss: 0.2706 - main_output_loss: 0.2656 - aux_output_loss: 0.3162
Epoch 3/200
363/363 [==============================] - 0s 492us/step - loss: 0.2732 - main_output_loss: 0.2682 - aux_output_loss: 0.3184
Epoch 4/200
363/363 [==============================] - 0s 478us/step - loss: 0.2726 - main_output_loss: 0.2674 - aux_output_loss: 0.3188
Epoch 5/200
363/363 [==============================] - 0s 464us/step - loss: 0.2718 - main_output_loss: 0.2667 - aux_output_loss: 0.3179
Epoch 6/200
363/363 [==============================] - 0s 478us/step - loss: 0.2710 - main_output_loss: 0.2660 - aux_output_loss: 0.3159
Epoch 7/200
363/363 [==============================] - 0s 492us/step - loss: 0.2725 - main_output_loss: 0.2674 - aux_output_loss: 0.3189
Epoch 8/200
363/363 [====================

363/363 [==============================] - 0s 456us/step - loss: 0.2708 - main_output_loss: 0.2657 - aux_output_loss: 0.3162
Epoch 24/200
363/363 [==============================] - 0s 450us/step - loss: 0.2709 - main_output_loss: 0.2658 - aux_output_loss: 0.3161
Epoch 25/200
363/363 [==============================] - 0s 450us/step - loss: 0.2701 - main_output_loss: 0.2650 - aux_output_loss: 0.3160
Epoch 26/200
363/363 [==============================] - 0s 445us/step - loss: 0.2682 - main_output_loss: 0.2631 - aux_output_loss: 0.3141
Epoch 27/200
363/363 [==============================] - 0s 453us/step - loss: 0.2703 - main_output_loss: 0.2651 - aux_output_loss: 0.3169
Epoch 28/200
363/363 [==============================] - 0s 450us/step - loss: 0.2702 - main_output_loss: 0.2652 - aux_output_loss: 0.3154
Epoch 29/200
363/363 [==============================] - 0s 453us/step - loss: 0.2712 - main_output_loss: 0.2661 - aux_output_loss: 0.3170
Epoch 30/200
363/363 [=========================

363/363 [==============================] - 0s 475us/step - loss: 0.2685 - main_output_loss: 0.2634 - aux_output_loss: 0.3137
Epoch 46/200
363/363 [==============================] - 0s 478us/step - loss: 0.2675 - main_output_loss: 0.2624 - aux_output_loss: 0.3129
Epoch 47/200
363/363 [==============================] - 0s 494us/step - loss: 0.2691 - main_output_loss: 0.2641 - aux_output_loss: 0.3139
Epoch 48/200
363/363 [==============================] - 0s 481us/step - loss: 0.2676 - main_output_loss: 0.2626 - aux_output_loss: 0.3130
Epoch 49/200
363/363 [==============================] - 0s 479us/step - loss: 0.2687 - main_output_loss: 0.2637 - aux_output_loss: 0.3135
Epoch 50/200
363/363 [==============================] - 0s 478us/step - loss: 0.2699 - main_output_loss: 0.2651 - aux_output_loss: 0.3134
Epoch 51/200
363/363 [==============================] - 0s 481us/step - loss: 0.2699 - main_output_loss: 0.2648 - aux_output_loss: 0.3155
Epoch 52/200
363/363 [=========================

363/363 [==============================] - 0s 452us/step - loss: 0.2676 - main_output_loss: 0.2626 - aux_output_loss: 0.3120
Epoch 68/200
363/363 [==============================] - 0s 452us/step - loss: 0.2659 - main_output_loss: 0.2610 - aux_output_loss: 0.3108
Epoch 69/200
363/363 [==============================] - 0s 459us/step - loss: 0.2689 - main_output_loss: 0.2641 - aux_output_loss: 0.3119
Epoch 70/200
363/363 [==============================] - 0s 477us/step - loss: 0.2676 - main_output_loss: 0.2627 - aux_output_loss: 0.3115
Epoch 71/200
363/363 [==============================] - 0s 448us/step - loss: 0.2664 - main_output_loss: 0.2617 - aux_output_loss: 0.3093
Epoch 72/200
363/363 [==============================] - 0s 461us/step - loss: 0.2664 - main_output_loss: 0.2615 - aux_output_loss: 0.3110
Epoch 73/200
363/363 [==============================] - 0s 459us/step - loss: 0.2655 - main_output_loss: 0.2606 - aux_output_loss: 0.3097
Epoch 74/200
363/363 [=========================

363/363 [==============================] - 0s 475us/step - loss: 0.2654 - main_output_loss: 0.2606 - aux_output_loss: 0.3094
Epoch 90/200
363/363 [==============================] - 0s 490us/step - loss: 0.2672 - main_output_loss: 0.2623 - aux_output_loss: 0.3113
Epoch 91/200
363/363 [==============================] - 0s 511us/step - loss: 0.2650 - main_output_loss: 0.2601 - aux_output_loss: 0.3091
Epoch 92/200
363/363 [==============================] - 0s 527us/step - loss: 0.2672 - main_output_loss: 0.2623 - aux_output_loss: 0.3113
Epoch 93/200
363/363 [==============================] - 0s 475us/step - loss: 0.2656 - main_output_loss: 0.2607 - aux_output_loss: 0.3096
Epoch 94/200
363/363 [==============================] - 0s 505us/step - loss: 0.2660 - main_output_loss: 0.2611 - aux_output_loss: 0.3104
Epoch 95/200
363/363 [==============================] - 0s 474us/step - loss: 0.2661 - main_output_loss: 0.2612 - aux_output_loss: 0.3097
Epoch 96/200
363/363 [=========================

363/363 [==============================] - 0s 466us/step - loss: 0.2660 - main_output_loss: 0.2611 - aux_output_loss: 0.3107
Epoch 112/200
363/363 [==============================] - 0s 459us/step - loss: 0.2638 - main_output_loss: 0.2591 - aux_output_loss: 0.3066
Epoch 113/200
363/363 [==============================] - 0s 478us/step - loss: 0.2655 - main_output_loss: 0.2606 - aux_output_loss: 0.3093
Epoch 114/200
363/363 [==============================] - 0s 470us/step - loss: 0.2648 - main_output_loss: 0.2600 - aux_output_loss: 0.3080
Epoch 115/200
363/363 [==============================] - 0s 472us/step - loss: 0.2671 - main_output_loss: 0.2625 - aux_output_loss: 0.3077
Epoch 116/200
363/363 [==============================] - 0s 470us/step - loss: 0.2652 - main_output_loss: 0.2603 - aux_output_loss: 0.3089
Epoch 117/200
363/363 [==============================] - 0s 493us/step - loss: 0.2649 - main_output_loss: 0.2601 - aux_output_loss: 0.3079
Epoch 118/200
363/363 [==================

363/363 [==============================] - 0s 469us/step - loss: 0.2631 - main_output_loss: 0.2584 - aux_output_loss: 0.3052
Epoch 134/200
363/363 [==============================] - 0s 461us/step - loss: 0.2627 - main_output_loss: 0.2579 - aux_output_loss: 0.3058
Epoch 135/200
363/363 [==============================] - 0s 453us/step - loss: 0.2643 - main_output_loss: 0.2596 - aux_output_loss: 0.3065
Epoch 136/200
363/363 [==============================] - 0s 459us/step - loss: 0.2642 - main_output_loss: 0.2595 - aux_output_loss: 0.3066
Epoch 137/200
363/363 [==============================] - 0s 467us/step - loss: 0.2666 - main_output_loss: 0.2620 - aux_output_loss: 0.3087
Epoch 138/200
363/363 [==============================] - 0s 471us/step - loss: 0.2671 - main_output_loss: 0.2628 - aux_output_loss: 0.3060
Epoch 139/200
363/363 [==============================] - 0s 461us/step - loss: 0.2642 - main_output_loss: 0.2596 - aux_output_loss: 0.3059
Epoch 140/200
363/363 [==================

363/363 [==============================] - 0s 493us/step - loss: 0.2623 - main_output_loss: 0.2576 - aux_output_loss: 0.3043
Epoch 156/200
363/363 [==============================] - 0s 515us/step - loss: 0.2615 - main_output_loss: 0.2568 - aux_output_loss: 0.3036
Epoch 157/200
363/363 [==============================] - 0s 545us/step - loss: 0.2618 - main_output_loss: 0.2570 - aux_output_loss: 0.3044
Epoch 158/200
363/363 [==============================] - 0s 483us/step - loss: 0.2624 - main_output_loss: 0.2576 - aux_output_loss: 0.3057
Epoch 159/200
363/363 [==============================] - 0s 479us/step - loss: 0.2621 - main_output_loss: 0.2575 - aux_output_loss: 0.3037
Epoch 160/200
363/363 [==============================] - 0s 483us/step - loss: 0.2624 - main_output_loss: 0.2578 - aux_output_loss: 0.3042
Epoch 161/200
363/363 [==============================] - 0s 542us/step - loss: 0.2631 - main_output_loss: 0.2585 - aux_output_loss: 0.3042
Epoch 162/200
363/363 [==================

363/363 [==============================] - 0s 462us/step - loss: 0.2617 - main_output_loss: 0.2571 - aux_output_loss: 0.3030
Epoch 178/200
363/363 [==============================] - 0s 453us/step - loss: 0.2620 - main_output_loss: 0.2575 - aux_output_loss: 0.3022
Epoch 179/200
363/363 [==============================] - 0s 455us/step - loss: 0.2660 - main_output_loss: 0.2613 - aux_output_loss: 0.3087
Epoch 180/200
363/363 [==============================] - 0s 469us/step - loss: 0.2625 - main_output_loss: 0.2577 - aux_output_loss: 0.3050
Epoch 181/200
363/363 [==============================] - 0s 467us/step - loss: 0.2609 - main_output_loss: 0.2561 - aux_output_loss: 0.3040
Epoch 182/200
363/363 [==============================] - 0s 453us/step - loss: 0.2628 - main_output_loss: 0.2580 - aux_output_loss: 0.3056
Epoch 183/200
363/363 [==============================] - 0s 464us/step - loss: 0.2627 - main_output_loss: 0.2581 - aux_output_loss: 0.3045
Epoch 184/200
363/363 [==================

363/363 [==============================] - 0s 458us/step - loss: 0.2618 - main_output_loss: 0.2571 - aux_output_loss: 0.3036
Epoch 200/200
363/363 [==============================] - 0s 456us/step - loss: 0.2613 - main_output_loss: 0.2566 - aux_output_loss: 0.3032


**Earlystopping** 콜백함수를 이용하면 일정 에포크동안 검증 세트에 대한 점수가 향상되지 않을 경우 학습을 멈출 수 있음.  
모델이 향상되지 않으면 학습을 자동으로 멈추기 때문에 에포크 숫자를 크게 지정해도 됨. 이 경우 학습이 끝나고 최상의 가중치를 복원해주기 때문에 저장된 모델을 따로 복원할 필요가 없음.  


더 많은 제어를 원한다면 **사용자 정의 콜백 함수** 를 만들 수도 있음.  
예를 들어 학습하는 동안 검증 손실과 학습 손실의 비율을 출력하는 콜백을 만들어 보겠음.

In [27]:
class PrintValTrainRatioCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        print("\nval/train: {:.2f}".format(logs["val_loss"] / logs["loss"]))

예상할 수 있듯이  
- **on_train_begin , on_train_end, on_epoch_begin, on_epoch_end, on_batch_begin, on_batch_end** 를 구현할 수 있음.  
  
또한 검증과 예측 단계에서도 사용할 수 있음(evaluate에서 사용됨).  
- **on_test_begin, on_test_end, on_test_batch_begin, on_test_batch_end** 를 구현하면 됨.  
  
예측에 사용하려면(predict에서 사용됨)  
- **on_predict_begin, on_predict_end, on_predict_batch_begin, on_predict_batch_end** 를 구현하면 됨

In [28]:
temp_cb = PrintValTrainRatioCallback()
hist = model3.fit((x_train_a, x_train_b), y_train, epochs=200,
                  validation_data=((x_val_a, x_val_b), y_val),
                callbacks=[checkpoint_cb, early_stopping_cb, temp_cb])

Epoch 1/200
363/363 [==============================] - 0s 705us/step - loss: 0.2530 - main_output_loss: 0.2488 - aux_output_loss: 0.2906 - val_loss: 0.2923 - val_main_output_loss: 0.2872 - val_aux_output_loss: 0.3387

val/train: 1.16
Epoch 2/200
363/363 [==============================] - 0s 630us/step - loss: 0.2563 - main_output_loss: 0.2521 - aux_output_loss: 0.2941 - val_loss: 0.3100 - val_main_output_loss: 0.3040 - val_aux_output_loss: 0.3638

val/train: 1.21
Epoch 3/200
363/363 [==============================] - 0s 636us/step - loss: 0.2535 - main_output_loss: 0.2493 - aux_output_loss: 0.2914 - val_loss: 0.2974 - val_main_output_loss: 0.2922 - val_aux_output_loss: 0.3442

val/train: 1.17
Epoch 4/200
363/363 [==============================] - 0s 677us/step - loss: 0.2549 - main_output_loss: 0.2507 - aux_output_loss: 0.2929 - val_loss: 0.2995 - val_main_output_loss: 0.2944 - val_aux_output_loss: 0.3459

val/train: 1.18
Epoch 5/200
363/363 [==============================] - 0s 647us/